In [11]:
import pandas as pd
from pandas.tseries.holiday import *
import numpy as np
from datetime import date

In [12]:
cal = USFederalHolidayCalendar()

In [13]:
cal.holidays()

DatetimeIndex(['1970-01-01', '1970-02-16', '1970-05-25', '1970-07-03',
               '1970-09-07', '1970-10-12', '1970-11-11', '1970-11-26',
               '1970-12-25', '1971-01-01',
               ...
               '2030-01-01', '2030-01-21', '2030-02-18', '2030-05-27',
               '2030-07-04', '2030-09-02', '2030-10-14', '2030-11-11',
               '2030-11-28', '2030-12-25'],
              dtype='datetime64[ns]', length=594, freq=None)

In [15]:
holidaylist = list(cal.holidays())
holidaylist[0].to_datetime() + DateOffset(days=1)
# [x.to_datetime() for x in list(cal.holidays())]


Timestamp('1970-01-02 00:00:00')

In [ ]:
def DaysUntilWorkDay(dt, calendar):
    holidays = [x.to_datetime() for x in list(calendar.holidays())]
    offset = 0
    offsetday = dt + DateOffset(days=offset)
    
    while offsetday.weekday() in [5,6] or offsetday in holidays:
        offset += 1
        offsetday += DateOffset(days=offset)
    
    return offset

DaysUntilWorkDay(Timestamp('2015-12-25'), cal)

In [ ]:
def WorkdaysNextWeek(dt, calendar):
    pass

In [ ]:
import holidays
from datetime import date

In [ ]:
us_holidays = holidays.US()

In [ ]:
us_holidays.get(date(2012 ,1, 2))

In [ ]:
for date, name in sorted(holidays.US(state='CA', years=[2012, 2013]).items()):
    if name == 'Thanksgiving':
        print date


## use holidays module
Need to create a DB table to support holidays

V1 schema - date and holiday booleans
* Date
* NewYearsDay
* US_MLK
* US_Washington
* CA_Victoria
* etc.

Store observed days if available

V2 schema - date, holiday booleans
* Date
* holidays...
* US_DaysUntilDayOff
* US_WorkdaysNextWeek
* US_DaysUntilWorkday
* CA_DaysUntilDayOff
* CA_WorkdaysNextWeek
* CA_DaysUntilWorkday

## V2
1. set day after Thanksgiving as another day of Thanksgiving
2. create features

In [296]:
from dbhelper import pd_query, PgDB
import re

In [297]:
query = '''
        select *, extract(dow from date) as dow
        from publicholiday
        where date < '2017-2-1'
        order by date desc
        '''

df = pd_query(query)

In [298]:
df.head(6)

,date,newyears,labor,us_mlk,us_washington,us_memorial,us_independence,us_columbus,us_veterans,us_thanksgiving,xmas,ca_goodfriday,ca_victoria,ca_canada,ca_civic,ca_thanksgiving,ca_boxing,ca_family,dow
0,2016-12-31,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,6
1,2016-12-30,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,5
2,2016-12-29,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,4
3,2016-12-28,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,3
4,2016-12-27,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,2
5,2016-12-26,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,1


In [299]:
ush = [val for val in df.columns.values if re.match(r'^ca_', val) is None and val != 'date' and val!= 'dow']
cah = [val for val in df.columns.values if re.match(r'^us_', val) is None and val != 'date' and val!= 'dow']

In [300]:
df['us_workday'] = (~df[ush].sum(1).astype(bool)) & ((df.dow < 6) & (df.dow > 0))
df['ca_workday'] = (~df[cah].sum(1).astype(bool)) & ((df.dow < 6) & (df.dow > 0))

In [301]:
trun = 0
frun = 0

runs = np.zeros((len(df), 2))

for (i, val) in enumerate(df.us_workday.values):
    if val:
        trun += 1
        frun = 0
    else:
        frun += 1
        trun = 0
        
    runs[i] = [trun, frun]

df['us_days_to_dayoff'] = runs[:,0]
df['us_days_to_workday'] = runs[:,1]

In [307]:
## Thursday holidays which can be spanned for long weekend
df[(df.dow==4) & (df.us_days_to_dayoff==0) & (df.us_days_to_workday==1)]

,date,newyears,labor,us_mlk,us_washington,us_memorial,us_independence,us_columbus,us_veterans,us_thanksgiving,...,ca_thanksgiving,ca_boxing,ca_family,dow,us_workday,ca_workday,us_days_to_dayoff,us_days_to_workday,ca_days_to_dayoff,ca_days_to_workday
730,2015-01-01,True,False,False,False,False,False,False,False,False,...,False,False,False,4,False,False,0,1,0,1
737,2014-12-25,False,False,False,False,False,False,False,False,False,...,False,False,False,4,False,False,0,1,0,4
1276,2013-07-04,False,False,False,False,False,True,False,False,False,...,False,False,False,4,False,True,0,1,2,0
2242,2010-11-11,False,False,False,False,False,False,False,True,False,...,False,False,False,4,False,True,0,1,2,0
2921,2009-01-01,True,False,False,False,False,False,False,False,False,...,False,False,False,4,False,False,0,1,0,1
2928,2008-12-25,False,False,False,False,False,False,False,False,False,...,False,False,False,4,False,False,0,1,0,4


In [ ]:
## Tu

In [303]:
df.head(60)

,date,newyears,labor,us_mlk,us_washington,us_memorial,us_independence,us_columbus,us_veterans,us_thanksgiving,...,ca_canada,ca_civic,ca_thanksgiving,ca_boxing,ca_family,dow,us_workday,ca_workday,us_days_to_dayoff,us_days_to_workday
0,2016-12-31,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,6,False,False,0,1
1,2016-12-30,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,5,True,True,1,0
2,2016-12-29,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,4,True,True,2,0
3,2016-12-28,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,3,True,True,3,0
4,2016-12-27,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,2,True,False,4,0
5,2016-12-26,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,1,False,False,0,1
6,2016-12-25,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,0,False,False,0,2
7,2016-12-24,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,6,False,False,0,3
8,2016-12-23,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,5,True,True,1,0
9,2016-12-22,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,4,True,True,2,0


In [304]:
trun = 0
frun = 0

runs = np.zeros((len(df), 2))

for (i, val) in enumerate(df.ca_workday.values):
    if val:
        trun += 1
        frun = 0
    else:
        frun += 1
        trun = 0
        
    runs[i] = [trun, frun]

df['ca_days_to_dayoff'] = runs[:,0]
df['ca_days_to_workday'] = runs[:,1]

In [198]:
df.head()

,date,newyears,labor,us_mlk,us_washington,us_memorial,us_independence,us_columbus,us_veterans,us_thanksgiving,...,ca_boxing,ca_family,us_thanksgivingdayafter,dow,us_workday,ca_workday,us_days_to_dayoff,us_days_to_workday,ca_days_to_dayoff,ca_days_to_workday
0,2016-12-31,False,False,False,False,False,False,False,False,False,...,False,False,False,6,True,True,1,0,1,0
1,2016-12-30,False,False,False,False,False,False,False,False,False,...,False,False,False,5,True,True,2,0,2,0
2,2016-12-29,False,False,False,False,False,False,False,False,False,...,False,False,False,4,True,True,3,0,3,0
3,2016-12-28,False,False,False,False,False,False,False,False,False,...,False,False,False,3,True,True,4,0,4,0
4,2016-12-27,False,False,False,False,False,False,False,False,False,...,True,False,False,2,True,True,5,0,5,0


## Find number of days in next workweek

In [199]:
df = df.sort_values('date')

In [200]:
df.head()

,date,newyears,labor,us_mlk,us_washington,us_memorial,us_independence,us_columbus,us_veterans,us_thanksgiving,...,ca_boxing,ca_family,us_thanksgivingdayafter,dow,us_workday,ca_workday,us_days_to_dayoff,us_days_to_workday,ca_days_to_dayoff,ca_days_to_workday
3652,2007-01-01,True,False,False,False,False,False,False,False,False,...,False,False,False,1,True,True,3653,0,3653,0
3651,2007-01-02,False,False,False,False,False,False,False,False,False,...,False,False,False,2,True,True,3652,0,3652,0
3650,2007-01-03,False,False,False,False,False,False,False,False,False,...,False,False,False,3,True,True,3651,0,3651,0
3649,2007-01-04,False,False,False,False,False,False,False,False,False,...,False,False,False,4,True,True,3650,0,3650,0
3648,2007-01-05,False,False,False,False,False,False,False,False,False,...,False,False,False,5,True,True,3649,0,3649,0


In [203]:
us_upcoming_workdays = []
for row in df.iterrows():
    dt = row[1][0]
#     print dt
    temp = df[(df.date > dt) & (df.us_workday == False)]
    temp = df[(df.date > temp.head(1).date.values[0]) & (df.us_workday == True)]
#     print temp.head(1).us_days_to_dayoff.values[0]
    us_upcoming_workdays.append(temp.head(1).us_days_to_dayoff.values[0])
    
#     break

IndexError: index 0 is out of bounds for axis 0 with size 0

In [202]:
len(us_upcoming_workdays)

0